In [1]:
import numpy as np
import tensorflow_addons as tfa

import tensorflow as tf
# from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers
AUTOTUNE = tf.data.experimental.AUTOTUNE


def get_label(file_path):
    parts = tf.strings.split(file_path, '/')
    return parts[-2] == np.array(['0', '1'])


def decode_img(img):
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    return img


def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)

    return img, label


list_ds = tf.data.Dataset.list_files("data/data/*/*")
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

DATASET_SIZE = 17000

train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = labeled_ds.shuffle(5000)
train_dataset = full_dataset.take(train_size).batch(32)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size).batch(32)
test_dataset = test_dataset.take(test_size).batch(32)
IMAGE_SIZE = 224

head = tf.keras.Sequential()
head.add(layers.Conv2D(32, (3, 3), input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
head.add(layers.BatchNormalization())
head.add(layers.Activation('relu'))
head.add(layers.MaxPooling2D(pool_size=(2, 2)))

head.add(layers.Conv2D(32, (3, 3)))
head.add(layers.BatchNormalization())
head.add(layers.Activation('relu'))
head.add(layers.MaxPooling2D(pool_size=(2, 2)))

head.add(layers.Conv2D(64, (3, 3)))
head.add(layers.BatchNormalization())
head.add(layers.Activation('relu'))
head.add(layers.MaxPooling2D(pool_size=(2, 2)))

average_pool = tf.keras.Sequential()
average_pool.add(layers.AveragePooling2D())
average_pool.add(layers.Flatten())
average_pool.add(layers.Dense(2, activation='softmax'))

standard_model = tf.keras.Sequential([
    head,
    average_pool
])


standard_model.compile(optimizer=tf.keras.optimizers.Adam(),
                       loss='binary_crossentropy',
                       metrics=['accuracy', tfa.metrics.F1Score(num_classes=2, average=None)])
callbacks = [tf.keras.callbacks.TensorBoard(
    log_dir='./log/standard_model', update_freq='batch')]

standard_model.fit(train_dataset, epochs=7,
                   validation_data=val_dataset, validation_steps=10, verbose=2, callbacks=callbacks)

score = standard_model.evaluate(test_dataset)
print(score)

Epoch 1/7
372/372 - 91s - loss: 2.2579 - accuracy: 0.8508 - f1_score: 0.4532 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 2/7
372/372 - 36s - loss: 2.2807 - accuracy: 0.8518 - f1_score: 0.4457 - val_loss: 1.5860 - val_accuracy: 0.8969 - val_f1_score: 0.4473
Epoch 3/7
372/372 - 37s - loss: 1.7321 - accuracy: 0.8395 - f1_score: 0.3562 - val_loss: 2.7725 - val_accuracy: 0.4594 - val_f1_score: 0.0000e+00
Epoch 4/7
372/372 - 43s - loss: 0.4055 - accuracy: 0.8453 - f1_score: 0.0000e+00 - val_loss: 0.6922 - val_accuracy: 0.6031 - val_f1_score: 0.0000e+00
Epoch 5/7
372/372 - 35s - loss: 0.3865 - accuracy: 0.8473 - f1_score: 9.9049e-05 - val_loss: 0.3624 - val_accuracy: 0.8406 - val_f1_score: 0.0000e+00
Epoch 6/7
372/372 - 36s - loss: 0.3322 - accuracy: 0.8636 - f1_score: 0.0000e+00 - val_loss: 0.2896 - val_accuracy: 0.8719 - val_f1_score: 0.0000e+00
Epoch 7/7
372/372 - 38s - loss: 0.3152 - accuracy: 0.8684 - f1_score: 0.0000e+00 - val_loss: 0.3298 - val_ac

In [2]:
list_ds = tf.data.Dataset.list_files("data/test/*")

In [3]:
def process_test_path(file_path):
    img = tf.io.read_file(file_path)
    img = decode_img(img)

    return img, [True, False]
test_ds = list_ds.map(process_test_path, num_parallel_calls=AUTOTUNE)

In [4]:
train_dataset

<DatasetV1Adapter shapes: ((None, None, None, 3), (None, 2)), types: (tf.float32, tf.bool)>

In [5]:
test_ds.batch(32)

<DatasetV1Adapter shapes: ((None, None, None, 3), (None, 2)), types: (tf.float32, tf.bool)>

In [6]:
y = standard_model.predict(test_ds.batch(32))

In [8]:
import pandas as pd

In [10]:
ans_df = pd.DataFrame({'id': [f"{'0'* (6-len(str(x)))}{x}" for x in range(17001,22150)], 
                              'class': np.argmax(y, axis=1)})

In [13]:
ans_df.to_csv('ans.csv', index=False)